# MegaDetector

In [27]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from PytorchWildlife.models import detection as pw_detection

def detect_objects(image_path):
    """
    Detects objects in an image using the MegaDetector model.

    Args:
        image_path (str): Path to the image file.

    Returns:
        dict: A dictionary containing the detection results, with keys:
            "detections": A list of dictionaries, where each dictionary
                          represents a detected object and contains the keys:
                          "bbox": Bounding box coordinates [x1, y1, x2, y2].
                          "confidence": Confidence score of the detection.
                          "category": Category ID of the detected object.
        None: If an error occurs during detection.
    """
    try:
        # Load the MegaDetector model
        detection_model = pw_detection.MegaDetectorV6(version="MDV6-yolov9-c")

        # Load the image
        img = np.array(Image.open(image_path))

        # Run object detection
        detection_results = detection_model.single_image_detection(img)

        # Prepare the output
        detections = []
        for i in range(len(detection_results["detections"].xyxy)):
            box = detection_results["detections"].xyxy[i]
            confidence = detection_results["detections"].confidence[i]
            category = detection_results["detections"].class_id[i]

            detections.append({
                "bbox": box.tolist(),
                "confidence": float(confidence),
                "category": int(category)
            })

        return {"detections": detections}

    except Exception as e:
        print(f"Error during object detection: {str(e)}")
        return None

def visualize_detections(image_path, detections, output_path):
    """
    Draws bounding boxes on an image and saves the result.

    Args:
        image_path (str): Path to the input image.
        detections (dict): A dictionary containing detection results
                           (as returned by detect_objects).
        output_path (str): Path to save the output image with bounding boxes.
    """
    fig = None
    try:
        # Load the image
        img = np.array(Image.open(image_path))

        # Create a figure and axes
        fig, ax = plt.subplots(1)
        ax.imshow(img)

        # Draw bounding boxes
        for detection in detections["detections"]:
            bbox = detection['bbox']
            confidence = detection['confidence']
            x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
            rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(x1, y1 - 5, f'{confidence:.2f}', color='white', fontsize=8, backgroundcolor='r')

        # Save the image
        plt.savefig(output_path)

    except Exception as e:
        print(f"Error during visualization: {str(e)}")
    finally:
        if fig:
            plt.close(fig)


# Example usage
image_path = 'D:/orinoquia_camera_traps_images/A09/100EK113/01130114.JPG'  # Replace with your image path
output_path = "detection_output.jpg"

if os.path.exists(image_path):
    detections = detect_objects(image_path)
    if detections:
        visualize_detections(image_path, detections, output_path)
        print(f"Detection results saved to {output_path}")
    else:
        print("No objects detected or an error occurred during detection.")
else:
    print(f"Image not found: {image_path}")

Ultralytics 8.3.74  Python-3.11.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
YOLOv9c summary (fused): 384 layers, 25,321,561 parameters, 0 gradients, 102.3 GFLOPs

0: 480x640 1 animal, 440.9ms
Speed: 4.0ms preprocess, 440.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Detection results saved to detection_output.jpg


## Old versions

In [26]:
import os
import json
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from PytorchWildlife.models import detection as pw_detection
import torch

def debug_pytorchwildlife_output(detection_results):
    print("Debugging PytorchWildlife Output:")
    print(f"Type of detection_results: {type(detection_results)}")
    if isinstance(detection_results, dict):
        if "detection_boxes" in detection_results and "detection_scores" in detection_results and "detection_classes" in detection_results:
            boxes = detection_results["detection_boxes"]
            scores = detection_results["detection_scores"]
            classes = detection_results["detection_classes"]

            print(f"Number of boxes: {len(boxes)}")
            print(f"Number of scores: {len(scores)}")
            print(f"Number of classes: {len(classes)}")
            for i in range(len(boxes)):
                print(f"--- Detection {i + 1} ---")
                print(f"Type of box: {type(boxes[i])}")
                print(f"Value of box: {boxes[i]}")
                print(f"Type of confidence: {type(scores[i])}")
                print(f"Value of confidence: {scores[i]}")
                print(f"Type of class: {type(classes[i])}")
                print(f"Value of class: {classes[i]}")

        else:
            print("Expected keys 'detection_boxes', 'detection_scores', and 'detection_classes' not found in detection_results.")
    else:
        print("Unexpected structure for detection_results. Not a dict.")

def run_megadetector_on_image(image_path):
    detection_model = pw_detection.MegaDetectorV6(version="MDV6-yolov9-c") # Model weights are automatically downloaded.

    img = np.array(Image.open(image_path))
    detection_results = detection_model.single_image_detection(img)

    print(f"detection_results: {detection_results}")
    #This is the section you must replace
    #DEBUG CODE
    # try:
    #     for i in range(len(detection_results["detections"].xyxy)):
    #         print(f"Boudning Boxes: {str(i)}")
    #     box = detection_results["detections"].xyxy[i]
    #     confidence = detection_results["detections"].confidence[i] #Sets parameters
    #     category = detection_results["detections"].class_id[i]
    #     print(f"Boudning box success: {box} confidence : {confidence}")
    # except Exception as e:
    #     print ("Object has no boudning box data, but the image is being tested (please make sure the format is correct for all")
    
    debug_pytorchwildlife_output(detection_results) # Debug the output

    detections = {}
    detections["images"] = {}
    detections["images"][image_path] = []

    # Format the results as MegaDetector output -Use XYXY Format for the image to use (It worked)
    try:
       for i in range(len(detection_results["detections"].xyxy)): #Try to view all bounding boxes to see if image is working

       #Set the bounding box object to be worked on
            box = detection_results["detections"].xyxy[i]
            confidence = detection_results["detections"].confidence[i] #Sets parameters
            category = detection_results["detections"].class_id[i]

           #Verify all images
            print("Set for images for  {image_path} : {i}")

           #Assign properties, can also move the transform here
            height, width, _ = img.shape #Sets proper height settings with image properties
            x1 = int(box[0]) #First coordinate for width point
            y1 = int(box[1]) #First ordinate for height point
            x2 = int(box[2]) #Second coordinate for Width point
            y2 = int(box[3]) #Second ordinate for height point

            print ("Proper params set:{i}") #Code that should be tested for

           #Append all variables (to local list)
            detections["images"][image_path].append({
                "bbox": box.tolist(),  # Convert box (torch.Tensor) to a list
                "confidence": float(confidence),  # Convert confidence (torch.Tensor) to a float
                "category": int(category),  # Convert category to int
            })
    except Exception as e: #Code breaks here if image file is bad
       print(f"Error processing detections: {str(e)}")

    return detections #Returns all image parameters

# Helper function to draw bounding boxes on images
def draw_boxes(image_path, detections, output_path):
    fig = None #Ensures the image saves with the matplotlib backend
    try:
        img = np.array(Image.open(image_path))
        print("Here is my images: {img}")
        #Scale bounding box coordinates from [y1, x1, y2, x2] to image size
        height, width, _ = img.shape
        print ("Here is is the width of the image, with shape{img.shape} for a height of {height} and width of {width}")
        if(image_path in detections["images"]):
            print ("Has passed file integrity checks to draw")
            
            fig,ax = plt.subplots(1)   #Assign function as a variable
            ax.imshow(img)  #Prints on graph
            
            for detection in detections["images"][image_path]: #Run on all images
                print ("Can finally draw the rectangle (after several steps)")
                bbox = detection['bbox']#Sets each part
                confidence = detection['confidence'] #Sets confidence levels
                x1 = int(bbox[0]) #x1
                y1 = int(bbox[1]) #y2
                x2 = int(bbox[2]) #x2
                y2 = int(bbox[3]) #x2 - All of it reads proper!
                rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')#Creates the boundary box

                ax.add_patch(rect) #Adds shape as point in the image
                ax.text(x1, y1-5, f'{confidence:.2f}', color = 'white', fontsize=8, backgroundcolor = 'r') #Add all text to the data
                #Show what data that is sent to the local document

                print ("Created Rectangle with parameters {x1} for x1, {y1} for y1, {x2} for x2, {y2} for y2, and a confidence of {confidence} with a file height of {height} and width of {width}")
    #This code should now all be completely fixed. This prints what will be placed into the file to ensure data transfer and a great outcome for project
            print ("End Rectangle")

        plt.savefig(output_path) #Saves the file

    except Exception as e: #Check code
        print (f"File may have broken, but is at least tested : {str(e)}")
    finally:
        if fig:
           plt.close(fig)

# Image name to check
# image_path = 'D:/orinoquia_camera_traps_images/A09/100EK113/01130114.JPG' #Base Image file
image_path = 'D:/orinoquia_camera_traps_images/A09/100EK113/01210144.JPG' #Base Image file

#Set Output Settings
output_path = "detection_output.jpg"

#These parameters may no longer exist, they may need to create functions to add them

#Make sure image path is valid
if os.path.exists(image_path): #If it exists then all code will work
    print ("This image path exists: {image_path}")
    try:
        detections = run_megadetector_on_image(image_path) #Add files to code (as we said)
        draw_boxes(image_path, detections, output_path) #Run code and generate boundary points

        print ("There were successful bounding boxes for {image_path} at the same directory")#Success

    except Exception as e:  #If broken check these steps to improve the code
        print (f"There are potential issues with library files: {str(e)}")
#The image did not exist skip
else:
    print (f"The image {image_path} does not exists, skipped")

This image path exists: {image_path}
Ultralytics 8.3.74  Python-3.11.2 torch-2.6.0+cpu CPU (Intel Core(TM) i7-10750H 2.60GHz)
YOLOv9c summary (fused): 384 layers, 25,321,561 parameters, 0 gradients, 102.3 GFLOPs

0: 480x640 1 animal, 402.0ms
Speed: 4.0ms preprocess, 402.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
detection_results: {'img_id': 'None', 'detections': Detections(xyxy=array([[     922.37,      118.56,        1920,      1337.7]], dtype=float32), mask=None, confidence=array([    0.97146], dtype=float32), class_id=array([0]), tracker_id=None, data={}), 'labels': ['animal 0.97']}
Debugging PytorchWildlife Output:
Type of detection_results: <class 'dict'>
Expected keys 'detection_boxes', 'detection_scores', and 'detection_classes' not found in detection_results.
Set for images for  {image_path} : {i}
Proper params set:{i}
Here is my images: {img}
Here is is the width of the image, with shape{img.shape} for a height of {height} and width of {width}
Has pa